In [ ]:
import pyopencl as cl
import numpy as np
import numpy.linalg as la

mf = cl.mem_flags

In [ ]:
import pyopencl.array

In [ ]:
ctx = cl.create_some_context(interactive=True)

queue = cl.CommandQueue(ctx)

Before running the remainder of this notebook, ensure that `make` has been run so that the `model.cl` and `model.py` files exist.

In [ ]:
prg_filename = 'tentusscher_panfilov_2006_M_cell.cl'
#prg_filename = 'TT06_M_manual.cl'

with open(prg_filename, 'r') as f:
#with open('test.cl', 'r') as f:
    prg_str = f.read()
    
prg = cl.Program(ctx, prg_str).build()

In [ ]:
import tentusscher_panfilov_2006_M_cell as TT06

In [ ]:
parameters = TT06.init_parameter_values()
default_initial_state_values = TT06.init_state_values()
num_states = default_initial_state_values.shape[0]

In [ ]:
V_index = TT06.state_indices("V")

In [ ]:
num_cells = 4
states = np.zeros((num_states, num_cells), dtype=np.float64)

In [ ]:
d_states = cl.array.to_device(queue, states)
d_parameters = cl.array.to_device(queue, parameters)

In [ ]:
WORK_GROUP_SIZE = None

In [ ]:
%%time
num_timesteps = 2000

t_values = np.zeros(num_timesteps, dtype=np.float64)
V_trace = np.zeros((num_timesteps, num_cells), dtype=np.float64)

t = np.float64(0.0)
dt = np.float64(1E-2)
prg.init_state_values(queue, (num_cells,), None,#(WORK_GROUP_SIZE,), 
                      d_states.data, np.uint32(num_cells)
                     )
for i in range(num_timesteps):
    prg.GRL1(queue, (num_cells,), None,#(WORK_GROUP_SIZE,), 
                        d_states.data, t, dt, d_parameters.data, np.uint32(num_cells)
                       )
    t += dt
    
    t_values[i] = t
    V_trace[i] = d_states[V_index].get()

In [ ]:
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'

In [ ]:
plt.plot(t_values, V_trace)
plt.show()

In [ ]:
prg.kernel_names